In [99]:
import math
import numpy as np
import random
from collections import Counter
from scipy import stats
from tqdm import tqdm
from scipy import stats

In [100]:
from os import walk
mypath = '../data/'
filenames = next(walk(mypath), (None, None, []))[2] 

NUM_DIGITS = 1000000
NUM_RUNS = 1000
pi_cf_digits = []
for filename in tqdm(filenames):
    pi_digits = np.load(mypath+filename)
    pi_cf_digits.extend(pi_digits['arr_0'])
pi_cf_digits = pi_cf_digits[1:(NUM_DIGITS*NUM_RUNS) +1]
pi_cf_digits = pi_cf_digits[:math.floor((len(pi_cf_digits)/NUM_DIGITS))* NUM_DIGITS]
pi_cf_digits = [int(x) for x in pi_cf_digits]
#NUM_RUNS = len(pi_cf_digits)//NUM_DIGITS
len(pi_cf_digits)

100%|█████████████████████████████████████████████| 3/3 [00:31<00:00, 10.61s/it]


1000000000

In [101]:
len(pi_cf_digits)

1000000000

# Normalized distance

### Random Walk

In [102]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(6)]
for i in range(len(nums)):
    distribution[i%6] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/sum(inv_distribution) for i in inv_distribution]
moves = [(1*inv_distribution[0], 0, 0),
         (0, 1*inv_distribution[1], 0),
         (-1*inv_distribution[2], 0, 0),
         (0, -1*inv_distribution[3], 0),
         (0, 0, 1*inv_distribution[4]),
         (0, 0, -1*inv_distribution[5])]

In [103]:
moves

[(2.1779485141953954, 0, 0),
 (0, 4.8188472628280135, 0),
 (-7.948219369940639, 0, 0),
 (0, -11.388675163458192, 0),
 (0, 0, 15.0300477027423),
 (0, 0, -18.80472852277334)]

In [104]:
distribution

[0.4591476765783108,
 0.2075185091907509,
 0.1258143432454694,
 0.08780652583793146,
 0.06653338830172478,
 0.05317811415298853]

In [105]:
def dist_from_origin(coordinates):
    return math.sqrt((coordinates[0]**2) + (coordinates[1]**2) + (coordinates[2]**2))

In [106]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i][0], \
                            random_walk[-1][1] + moves[i][1], random_walk[-1][2] + moves[i][2]))
    distance = 0
    for i in random_walk[1:]:
        distance += dist_from_origin(i)
    avg_dist = distance/(NUM_DIGITS)
    return avg_dist

In [107]:
avg_distances = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3, 4, 5], weights = distribution, k = NUM_DIGITS)
    avg_distances.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1000/1000 [10:24<00:00,  1.60it/s]


In [108]:
stats.describe(avg_distances)

DescribeResult(nobs=1000, minmax=(1625.346993894233, 11850.834440678807), mean=4684.282160835811, variance=2522206.1785858697, skewness=0.8701215176095131, kurtosis=1.0268613549881973)

### $\pi$ Walk

In [109]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [110]:
def pi_walk(pi_digits):
    mod_6_digits = [(i-1)%6 for i in pi_digits]
    pi_walk = [(0, 0, 0)]
    for i in range(len(mod_6_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_6_digits[i]][0], pi_walk[-1][1] + moves[mod_6_digits[i]][1]\
                       , pi_walk[-1][2] + moves[mod_6_digits[i]][2]))
    total_distance = 0
    for i in range(len(pi_walk)):
        total_distance += dist_from_origin(pi_walk[i])
    return total_distance/(NUM_DIGITS)

In [111]:
avg_pi_distances = []
test = []
for i in tqdm(range(NUM_RUNS)):
    avg_pi_distances.append(pi_walk(groups[i]))


100%|███████████████████████████████████████| 1000/1000 [21:07<00:00,  1.27s/it]


In [112]:
stats.describe(avg_pi_distances)

DescribeResult(nobs=1000, minmax=(1380.0823643908398, 13175.605412158338), mean=4694.231554547194, variance=2976212.7692270693, skewness=1.0277336015417327, kurtosis=1.5886561419378884)

# Sites visited

### Random Walk

In [113]:
nums = [math.log(1+1/(k*(k+2)))/math.log(2) for k in range(1, NUM_DIGITS+1)]
distribution = [0 for i in range(6)]
for i in range(len(nums)):
    distribution[i%6] += nums[i]
inv_distribution = [1/distribution[i] for i in range(len(distribution))]
normalized_inv_distribution = [float(i)/sum(inv_distribution) for i in inv_distribution]
moves = [(1*normalized_inv_distribution[0], 0, 0),
         (0, 1*normalized_inv_distribution[1], 0),
         (0, 0, 1*normalized_inv_distribution[2]),
         (-1*normalized_inv_distribution[3], 0, 0),
         (0, -1*normalized_inv_distribution[4], 0),
         (0, 0, -1*normalized_inv_distribution[5])]

In [114]:
moves

[(0.036197507425164865, 0, 0),
 (0, 0.08008924840970935, 0),
 (0, 0, 0.13209941731177852),
 (-0.1892797975274287, 0, 0),
 (0, -0.24979941434547012, 0),
 (0, 0, -0.3125346149804484)]

In [115]:
def single_random_walk_run(random_dist):
    random_walk = [(0, 0, 0)]
    for i in rand_distribution:
        random_walk.append((random_walk[-1][0] + moves[i][0], \
                            random_walk[-1][1] + moves[i][1], random_walk[-1][2] + moves[i][2]))
    sites = set()
    for i in random_walk:
        sites.add((math.floor(i[0]), math.floor(i[1]), math.floor(i[2])))
    return len(sites)

In [116]:
sites_visited = []
for i in tqdm(range(NUM_RUNS)):
    rand_distribution = random.choices([0, 1, 2, 3, 4, 5], weights = distribution, k = NUM_DIGITS)
    sites_visited.append(single_random_walk_run(rand_distribution))

100%|███████████████████████████████████████| 1000/1000 [13:56<00:00,  1.20it/s]


In [117]:
stats.describe(sites_visited)

DescribeResult(nobs=1000, minmax=(17784, 21548), mean=20094.057, variance=207318.97872972972, skewness=-0.40178150837036014, kurtosis=0.6984989345841526)

### $\pi$ Walk

In [118]:
groups = list(np.array_split(pi_cf_digits, NUM_RUNS))

In [119]:
def pi_walk(pi_digits):
    mod_6_digits = [(i-1)%6 for i in pi_digits]
    pi_walk = [(0, 0, 0)]
    for i in range(len(mod_6_digits)):
        pi_walk.append((pi_walk[-1][0] + moves[mod_6_digits[i]][0], pi_walk[-1][1] + moves[mod_6_digits[i]][1]\
                       , pi_walk[-1][2] + moves[mod_6_digits[i]][2]))
    sites = set()
    for i in pi_walk:
        sites.add((math.floor(i[0]), math.floor(i[1]), math.floor(i[2])))
    return len(sites)

In [120]:
sites_visited_pi = []
for i in tqdm(range(NUM_RUNS)):
    sites_visited_pi.append(pi_walk(groups[i]))

100%|███████████████████████████████████████| 1000/1000 [18:41<00:00,  1.12s/it]


In [121]:
stats.describe(sites_visited_pi)

DescribeResult(nobs=1000, minmax=(17880, 21135), mean=19854.228, variance=193608.84285885887, skewness=-0.33934763823166925, kurtosis=0.7240743243515158)